# Notebook 15: Agentic Workflows - Building MCP Servers

**Learning Objectives:**
- Create reusable MCP servers
- Implement resources and prompts
- Connect multiple clients to one server
- Build production-ready tool interfaces

## Prerequisites

### Hardware Requirements

| Model Option | Model Name | Size | Min RAM | Recommended Setup | Notes |
|--------------|------------|------|---------|-------------------|-------|
| **small (CPU-friendly)** | llama3.2:1b | 1.3GB | 8GB | 8GB RAM, CPU | Fast, good for learning |
| **large (GPU-optimized)** | llama3.1:8b | 4.7GB | 16GB | 12GB VRAM (RTX 4080) | Better reasoning |
| **SOTA (reference only)** | Claude 3.5 Sonnet | API | N/A | API key required | Production-grade |

### Software Requirements
- Python 3.10+
- Completed Notebook 14 (MCP Basics)
- Libraries: `mcp`, `ollama`, `pydantic`, `aiofiles`

### Installation

```bash
pip install mcp ollama pydantic aiofiles
```

## MCP Server Architecture

In Notebook 14, we built tools inline. Now we'll create **proper MCP servers** that can be reused across multiple clients.

### Server Components

1. **Tools** - Functions the LLM can invoke
2. **Resources** - Data sources (files, APIs)
3. **Prompts** - Reusable prompt templates
4. **Server Class** - Manages tools/resources/prompts

### Why Build Servers?

- **Reusability** - Write once, use in any MCP client
- **Separation of Concerns** - Tools separate from LLM logic
- **Testability** - Test tools independently
- **Scalability** - Multiple clients can use same server

## Expected Behaviors

### Server Initialization
```
Initializing MCP Server: FileSystemServer
Registered 3 tools: read_file, write_file, list_files
Registered 1 resource: filesystem://
Server ready
```

### Tool Execution
- Server validates tool parameters
- Executes tool logic
- Returns structured results
- Handles errors gracefully

### Resource Access
- Resources are URI-based (e.g., `file://path/to/file`)
- Support streaming for large resources
- Automatic caching where appropriate

### Performance
- Tool execution: 10-100ms (file operations)
- Server overhead: <5ms per request
- Resource streaming: Handles large files efficiently

In [ ]:
import json
import os
import random
from pathlib import Path
from typing import Any, Optional
import ollama
from pydantic import BaseModel, Field
import warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
random.seed(1103)

print("MCP Server Tutorial - Setup Complete")

## Model Selection

In [ ]:
# CHOOSE YOUR MODEL:

# Option 1: small model (CPU-friendly)
MODEL_NAME = "llama3.2:1b"  # 1.3GB

# Option 2: large model (GPU-optimized)
# MODEL_NAME = "llama3.1:8b"  # 4.7GB

print(f"Selected model: {MODEL_NAME}")

## Building a File System MCP Server

Let's build a complete MCP server for file operations.

In [ ]:
class FileSystemServer:
    """MCP Server for file system operations."""
    
    def __init__(self, base_path: str = "./mcp_workspace"):
        self.base_path = Path(base_path)
        self.base_path.mkdir(exist_ok=True)
        print(f"FileSystemServer initialized")
        print(f"Base path: {self.base_path.absolute()}")
        
        # Define tools
        self.tools = [
            {
                'type': 'function',
                'function': {
                    'name': 'read_file',
                    'description': 'Read contents of a file',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'filename': {
                                'type': 'string',
                                'description': 'Name of the file to read'
                            }
                        },
                        'required': ['filename']
                    }
                }
            },
            {
                'type': 'function',
                'function': {
                    'name': 'write_file',
                    'description': 'Write content to a file',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'filename': {
                                'type': 'string',
                                'description': 'Name of the file to write'
                            },
                            'content': {
                                'type': 'string',
                                'description': 'Content to write to the file'
                            }
                        },
                        'required': ['filename', 'content']
                    }
                }
            },
            {
                'type': 'function',
                'function': {
                    'name': 'list_files',
                    'description': 'List all files in the workspace',
                    'parameters': {
                        'type': 'object',
                        'properties': {},
                        'required': []
                    }
                }
            },
            {
                'type': 'function',
                'function': {
                    'name': 'delete_file',
                    'description': 'Delete a file from the workspace',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'filename': {
                                'type': 'string',
                                'description': 'Name of the file to delete'
                            }
                        },
                        'required': ['filename']
                    }
                }
            }
        ]
        
        # Map function names to methods
        self.function_map = {
            'read_file': self.read_file,
            'write_file': self.write_file,
            'list_files': self.list_files,
            'delete_file': self.delete_file
        }
        
        print(f"Registered {len(self.tools)} tools")
    
    def _safe_path(self, filename: str) -> Path:
        """Ensure path is within base directory."""
        path = (self.base_path / filename).resolve()
        if not str(path).startswith(str(self.base_path.resolve())):
            raise ValueError("Path outside workspace")
        return path
    
    def read_file(self, filename: str) -> str:
        """Read file contents."""
        try:
            path = self._safe_path(filename)
            if not path.exists():
                return f"Error: File '{filename}' not found"
            with open(path, 'r', encoding='utf-8') as f:
                content = f.read()
            return f"File '{filename}' contents:\n{content}"
        except Exception as e:
            return f"Error reading file: {str(e)}"
    
    def write_file(self, filename: str, content: str) -> str:
        """Write content to file."""
        try:
            path = self._safe_path(filename)
            with open(path, 'w', encoding='utf-8') as f:
                f.write(content)
            return f"Successfully wrote {len(content)} characters to '{filename}'"
        except Exception as e:
            return f"Error writing file: {str(e)}"
    
    def list_files(self) -> str:
        """List all files in workspace."""
        try:
            files = [f.name for f in self.base_path.iterdir() if f.is_file()]
            if not files:
                return "Workspace is empty"
            return f"Files in workspace: {', '.join(files)}"
        except Exception as e:
            return f"Error listing files: {str(e)}"
    
    def delete_file(self, filename: str) -> str:
        """Delete a file."""
        try:
            path = self._safe_path(filename)
            if not path.exists():
                return f"Error: File '{filename}' not found"
            path.unlink()
            return f"Successfully deleted '{filename}'"
        except Exception as e:
            return f"Error deleting file: {str(e)}"
    
    def execute_tool(self, tool_name: str, arguments: dict) -> str:
        """Execute a tool by name."""
        if tool_name not in self.function_map:
            return f"Error: Unknown tool '{tool_name}'"
        function = self.function_map[tool_name]
        return function(**arguments)

print("FileSystemServer class defined")

In [ ]:
# Initialize the server
fs_server = FileSystemServer()

print(f"\nAvailable tools:")
for tool in fs_server.tools:
    print(f"  - {tool['function']['name']}: {tool['function']['description']}")

## Agent with File System Access

In [ ]:
def run_fs_agent(prompt: str, model: str = MODEL_NAME, max_iterations: int = 10) -> str:
    """
    Run an agent with file system access.
    """
    messages = [{'role': 'user', 'content': prompt}]
    
    print(f"\n{'='*70}")
    print(f"User: {prompt}")
    print(f"{'='*70}\n")
    
    for iteration in range(max_iterations):
        response = ollama.chat(
            model=model,
            messages=messages,
            tools=fs_server.tools
        )
        
        messages.append(response['message'])
        
        if not response['message'].get('tool_calls'):
            final_answer = response['message']['content']
            print(f"Agent: {final_answer}")
            print(f"\n{'='*70}")
            return final_answer
        
        for tool_call in response['message']['tool_calls']:
            function_name = tool_call['function']['name']
            function_args = tool_call['function']['arguments']
            
            print(f"Tool Call: {function_name}({function_args})")
            
            result = fs_server.execute_tool(function_name, function_args)
            print(f"Tool Result: {result}\n")
            
            messages.append({
                'role': 'tool',
                'content': str(result)
            })
    
    return "Maximum iterations reached"

print("File system agent ready")

## Example 1: Create and Read Files

In [ ]:
result = run_fs_agent(
    "Create a file called 'notes.txt' with the content 'Hello from MCP server!'"
)

In [ ]:
result = run_fs_agent("Read the file 'notes.txt'")

## Example 2: List and Manage Files

In [ ]:
result = run_fs_agent("What files are in the workspace?")

## Example 3: Multi-Step File Operations

In [ ]:
result = run_fs_agent(
    """Create three files:
    1. 'todo.txt' with content 'Buy groceries\nFinish homework'
    2. 'shopping.txt' with content 'Milk\nBread\nEggs'
    3. 'summary.txt' with content 'Created 2 list files'
    Then list all files in the workspace.
    """
)

## Building a Data Analysis Server

Let's create another server for data operations.

In [ ]:
class DataAnalysisServer:
    """MCP Server for data analysis operations."""
    
    def __init__(self):
        self.tools = [
            {
                'type': 'function',
                'function': {
                    'name': 'calculate_statistics',
                    'description': 'Calculate mean, median, min, max for a list of numbers',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'numbers': {
                                'type': 'array',
                                'items': {'type': 'number'},
                                'description': 'List of numbers to analyze'
                            }
                        },
                        'required': ['numbers']
                    }
                }
            },
            {
                'type': 'function',
                'function': {
                    'name': 'sort_data',
                    'description': 'Sort a list of numbers',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'numbers': {
                                'type': 'array',
                                'items': {'type': 'number'},
                                'description': 'List of numbers to sort'
                            },
                            'ascending': {
                                'type': 'boolean',
                                'description': 'Sort in ascending order (default: true)'
                            }
                        },
                        'required': ['numbers']
                    }
                }
            },
            {
                'type': 'function',
                'function': {
                    'name': 'count_values',
                    'description': 'Count occurrences of each value in a list',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'items': {
                                'type': 'array',
                                'items': {'type': 'string'},
                                'description': 'List of items to count'
                            }
                        },
                        'required': ['items']
                    }
                }
            }
        ]
        
        self.function_map = {
            'calculate_statistics': self.calculate_statistics,
            'sort_data': self.sort_data,
            'count_values': self.count_values
        }
        
        print(f"DataAnalysisServer initialized with {len(self.tools)} tools")
    
    def calculate_statistics(self, numbers: list) -> str:
        """Calculate statistics for a list of numbers."""
        if not numbers:
            return "Error: Empty list"
        
        sorted_nums = sorted(numbers)
        n = len(sorted_nums)
        
        mean = sum(numbers) / n
        median = sorted_nums[n // 2] if n % 2 == 1 else (sorted_nums[n // 2 - 1] + sorted_nums[n // 2]) / 2
        
        stats = {
            'count': n,
            'mean': round(mean, 2),
            'median': median,
            'min': min(numbers),
            'max': max(numbers),
            'sum': sum(numbers)
        }
        
        return json.dumps(stats, indent=2)
    
    def sort_data(self, numbers: list, ascending: bool = True) -> str:
        """Sort a list of numbers."""
        sorted_nums = sorted(numbers, reverse=not ascending)
        direction = "ascending" if ascending else "descending"
        return f"Sorted ({direction}): {sorted_nums}"
    
    def count_values(self, items: list) -> str:
        """Count occurrences of each value."""
        from collections import Counter
        counts = Counter(items)
        return json.dumps(dict(counts), indent=2)
    
    def execute_tool(self, tool_name: str, arguments: dict) -> str:
        """Execute a tool by name."""
        if tool_name not in self.function_map:
            return f"Error: Unknown tool '{tool_name}'"
        function = self.function_map[tool_name]
        return function(**arguments)

data_server = DataAnalysisServer()

## Multi-Server Agent

An agent that can use tools from multiple servers.

In [ ]:
# Combine tools from both servers
all_tools = fs_server.tools + data_server.tools

# Combine function maps
all_functions = {**fs_server.function_map, **data_server.function_map}

print(f"Combined server with {len(all_tools)} tools")
print("\nAvailable tools:")
for tool in all_tools:
    print(f"  - {tool['function']['name']}")

In [ ]:
def run_multi_server_agent(prompt: str, model: str = MODEL_NAME, max_iterations: int = 10) -> str:
    """
    Run an agent with access to multiple servers.
    """
    messages = [{'role': 'user', 'content': prompt}]
    
    print(f"\n{'='*70}")
    print(f"User: {prompt}")
    print(f"{'='*70}\n")
    
    for iteration in range(max_iterations):
        response = ollama.chat(
            model=model,
            messages=messages,
            tools=all_tools
        )
        
        messages.append(response['message'])
        
        if not response['message'].get('tool_calls'):
            final_answer = response['message']['content']
            print(f"Agent: {final_answer}")
            print(f"\n{'='*70}")
            return final_answer
        
        for tool_call in response['message']['tool_calls']:
            function_name = tool_call['function']['name']
            function_args = tool_call['function']['arguments']
            
            print(f"Tool Call: {function_name}({function_args})")
            
            # Route to correct server
            if function_name in fs_server.function_map:
                result = fs_server.execute_tool(function_name, function_args)
            elif function_name in data_server.function_map:
                result = data_server.execute_tool(function_name, function_args)
            else:
                result = f"Error: Unknown tool '{function_name}'"
            
            print(f"Tool Result: {result}\n")
            
            messages.append({
                'role': 'tool',
                'content': str(result)
            })
    
    return "Maximum iterations reached"

print("Multi-server agent ready")

## Example: Data Analysis with File Storage

In [ ]:
result = run_multi_server_agent(
    """Analyze these test scores: [85, 92, 78, 95, 88, 91, 87, 93]. 
    Calculate the statistics and save the results to a file called 'test_scores_analysis.txt'.
    """
)

In [ ]:
result = run_multi_server_agent("Read the file 'test_scores_analysis.txt'")

## Server Best Practices

### 1. Clear Tool Descriptions

In [ ]:
# Good: Clear, specific description
good_description = "Calculate mean, median, min, max for a list of numbers"

# Bad: Vague description
bad_description = "Do math stuff"

print("Good description:", good_description)
print("Bad description:", bad_description)
print("\nClear descriptions help LLMs choose the right tool!")

### 2. Error Handling

In [ ]:
# Test error handling
result = run_fs_agent("Read a file that doesn't exist called 'nonexistent.txt'")

### 3. Security Considerations

In [ ]:
# FileSystemServer prevents path traversal attacks
print("Testing security:")
try:
    # This should fail - trying to access parent directory
    result = fs_server.read_file("../../../etc/passwd")
    print(result)
except ValueError as e:
    print(f"Security check passed: {e}")

## Exercises

1. **New Server**: Create a `WebSearchServer` with a mock search tool
2. **Resource Support**: Add a resource to FileSystemServer for accessing files by URI
3. **Prompt Templates**: Add reusable prompts to a server
4. **Error Recovery**: Test how the agent handles tool failures
5. **Performance**: Measure tool execution times for different operations

In [ ]:
# Your code here for exercises


## Key Takeaways

✅ **MCP Servers** provide reusable tool interfaces

✅ **Multiple servers** can be combined for one agent

✅ **Clear descriptions** improve tool selection accuracy

✅ **Error handling** makes servers robust

✅ **Security** matters - validate inputs and paths

## Next Steps

- Try **Notebook 16**: Multi-Tool MCP Agents
- Explore [MCP servers on GitHub](https://github.com/modelcontextprotocol/servers)
- Build your own custom MCP server

## Resources

- [MCP Server Guide](https://modelcontextprotocol.io/docs/concepts/servers)
- [Building Servers](https://modelcontextprotocol.io/docs/building/servers)
- [MCP Examples](https://github.com/modelcontextprotocol/examples)
- [MCP Python SDK](https://github.com/modelcontextprotocol/python-sdk)